In [7]:
from pmu_som.pmu_calc import dac_to_i

CRF = 0.5
val = 2e-3*CRF
rsens = 500
MI_GAIN = 10
VREF = 4.95
DAC_C = round((((val) * (2**16) * rsens * MI_GAIN)/(4.5 * VREF)) + (2**15))

print(dac_to_i(DAC_C, '2m'))
print(dac_to_i(0xDAC_C, '2m'))

0.0010000229644775392
0.0015800784301757814


In [8]:
## Clamp High





from keithley2600 import Keithley2600
from pmu_som import communicator, pmu_ctrl, pmu_cal
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd


try:
    del(p.c)
except:
    pass

k = Keithley2600('GPIB0::17::INSTR', visa_library='')
#k.smua.source.output = k.smua.OUTPUT_OFF

k.smua.reset()
k.smub.reset()

# p = communicator.communicator(False)
p = pmu_ctrl.pmu()

#print(k.query("*IDN?"))

p.channels[0].change_pmu_reg({
    "CH EN" : 0
})



k.display.screen = k.display.SMUA
k.display.smua.measure.func = k.display.MEASURE_DCVOLTS


k.smua.measure.nplc = 1
k.smua.measure.autorangei = k.smua.AUTORANGE_ON
k.smua.measure.autorangev = k.smua.AUTORANGE_ON


k.smua.source.limiti = ((2e-3)*2)
k.smua.source.limitv = 20
k.smua.source.func = k.smua.OUTPUT_DCAMPS
k.smua.source.leveli = 0
k.smua.source.output = k.smua.OUTPUT_ON

# Configure the system control settings for the PMU
p.change_sys_ctrl({
    "DUTGND/CH": 1,
    "INT10K": 1,
    "GAIN": 2,
    "TMP ENABLE": 1,
    "TMP": 3  # Thermal shutdown at 100°C
})


# Enable and configure the first PMU channel
p.channels[0].change_pmu_reg({
    "CH EN": 1,  # Enable the channel
    "FORCE": 1,  # Force current clamp voltage
    "SS0": 1,
    "SF0": 1,
    "FIN": 1,
    "C": 3,
    "CL": 1
})

num = 10
overlap = 0.5
measure_points = np.linspace(round(0x8000-(0x8000*overlap)), 0xFFFF, num)

CRF = 0.5
val = 2e-3*CRF
rsens = 500
MI_GAIN = 10
VREF = 4.95

DAC_C = round((((val) * (2**16) * rsens * MI_GAIN)/(4.5 * VREF)) + (2**15))

measure_delay = 0.1


## This code is only for clamp high
## Writing the DAC code for 2mA v to FIN 1 2m

p.channels[0].write_dac(
    data = DAC_C,
    dac_reg = "FIN_I_2m",
    reg = "X1"
    
)

 ## Writing Clamp Low to zero

p.channels[0].write_dac(
    data = 0x0000,
    dac_reg = "CLL_U",
    reg = "X1"
)

dac_regi = "CLH_U"

clamp_measured_values = []
clamp_expected_values = []


for d in measure_points :
    p.channels[0].write_dac(
        data = round(d),
        dac_reg = dac_regi,
        reg = "X1"
    )
    sleep (measure_delay)
    clamp_measured = k.smua.measure.v()
    clamp_measured_values.append(clamp_measured)
    val = d
    offset_dac = 0xA492
    clamp_expected = ((4.5 * val) - (3.5 * offset_dac)) * (VREF / (2**16))
    clamp_expected_values.append(clamp_expected)
    
    print("Expected Clamping :",clamp_expected)
    print("Measured Clamping :",clamp_measured)
    sleep (measure_delay)


p.channels[0].change_pmu_reg({
    "CH EN" :0
})


k.smua.source.output = k.smua.OUTPUT_OFF
df = pd.DataFrame({
    "Measured Clamping " :  clamp_measured_values,
    "Expected Clamping " :  clamp_expected_values
})

 

print(df)
    

Expected Clamping : -5.568674468994141
Measured Clamping : -5.51633
Expected Clamping : -3.7124622344970706
Measured Clamping : -3.64341
Expected Clamping : -1.8562500000000002
Measured Clamping : -1.77019
Expected Clamping : -3.77655029296875e-05
Measured Clamping : 0.103391
Expected Clamping : 1.8561744689941406
Measured Clamping : 1.97676
Expected Clamping : 3.712386703491211
Measured Clamping : 3.84998
Expected Clamping : 5.568598937988281
Measured Clamping : 5.72357
Expected Clamping : 7.424811172485354
Measured Clamping : 7.59511
Expected Clamping : 9.281023406982422
Measured Clamping : 9.46784
Expected Clamping : 11.137235641479492
Measured Clamping : 11.341
   Measured Clamping   Expected Clamping 
0           -5.516330           -5.568674
1           -3.643410           -3.712462
2           -1.770190           -1.856250
3            0.103391           -0.000038
4            1.976760            1.856174
5            3.849980            3.712387
6            5.723570           

In [9]:
 pmu_cal.measureCL_U(p, k, 0, range = "2m", current_range_factor=0.5, clamp_side='H', overlap=0.5, num=10, measure_delay = 0.1)

(array([-5.56867447e+00, -3.71246223e+00, -1.85625000e+00, -3.77655029e-05,
         1.85617447e+00,  3.71238670e+00,  5.56859894e+00,  7.42481117e+00,
         9.28102341e+00,  1.11372356e+01]),
 array([-5.51621 , -3.64338 , -1.7702  ,  0.103397,  1.9767  ,  3.84993 ,
         5.72352 ,  7.59505 ,  9.46781 , 11.341   ]))

In [15]:

#     ## Clamp LOW Test 





# from keithley2600 import Keithley2600
# from pmu_som import communicator, pmu_ctrl, pmu_cal
# from time import sleep
# import numpy as np
# from matplotlib import pyplot as plt
# from tqdm import tqdm
# import pandas as pd


# try:
#     del(p.c)
# except:
#     pass

# k = Keithley2600('GPIB0::17::INSTR', visa_library='')
# #k.smua.source.output = k.smua.OUTPUT_OFF

# k.smua.reset()
# k.smub.reset()

# # p = communicator.communicator(False)
# p = pmu_ctrl.pmu()

# #print(k.query("*IDN?"))

# p.channels[0].change_pmu_reg({
#     "CH EN" : 0
# })
# p.write_all_PMU_REGS(0x1E060)


# k.display.screen = k.display.SMUA
# k.display.smua.measure.func = k.display.MEASURE_DCVOLTS


# k.smua.measure.nplc = 1
# k.smua.measure.autorangei = k.smua.AUTORANGE_ON
# k.smua.measure.autorangev = k.smua.AUTORANGE_ON


# k.smua.source.limiti = ((2e-3)*2)
# k.smua.source.limitv = 20
# k.smua.source.func = k.smua.OUTPUT_DCAMPS
# k.smua.source.levelv = 0
# k.smua.source.output = k.smua.OUTPUT_ON

# # Configure the system control settings for the PMU
# p.change_sys_ctrl({
#     "DUTGND/CH": 1,
#     "INT10K": 1,
#     "GAIN": 2,
#     "TMP ENABLE": 1,
#     "TMP": 3  # Thermal shutdown at 100°C
# })


# # Enable and configure the first PMU channel
# p.channels[0].change_pmu_reg({
#     "CH EN": 1,  # Enable the channel
#     "FORCE": 1,  # Force current clamp voltage
#     "SS0": 1,
#     "SF0": 1,
#     "FIN": 1,
#     "C": 3,
#     "CL": 1
# })

# num = 10
# overlap = 0.5
# measure_points = np.linspace(round(0x7FFF+(0x8000*overlap)), 0, num)

# CRF = 0.5
# val = (2e-3)*-1*CRF
# rsens = 500
# MI_GAIN = 10
# VREF = 4.95

# DAC_C = round(((val * (2**16) * rsens * MI_GAIN)/(4.5 * VREF)) + (2**15))

# measure_delay = 0.1


# ## This code is only for clamp high
# ## Writing the DAC code for 2mA v to FIN 1 2m

# p.channels[0].write_dac(
#     data = DAC_C,
#     dac_reg = "FIN_I_2m",
#     reg = "X1"
    
# )

#  ## Writing Clamp Low to zero

# p.channels[0].write_dac(
#     data = 0xFFFF,
#     dac_reg ="CLH_U" ,
#     reg = "X1"
# )

# dac_regi = "CLL_U"

# clamp_measured_values = []
# clamp_expected_values = []


# for d in measure_points :
#     p.channels[0].write_dac(
#         data = round(d),
#         dac_reg = dac_regi,
#         reg = "X1"
#     )
#     sleep (measure_delay)
#     clamp_measured = k.smua.measure.v()
#     clamp_measured_values.append(clamp_measured)
#     val = d
#     offset_dac = 0xA492
#     clamp_expected = ((4.5 * val) - (3.5 * offset_dac)) * (VREF / (2**16))
#     clamp_expected_values.append(clamp_expected)
    
#     print("Expected Clamping :",clamp_expected)
#     print("Measured Clamping :",clamp_measured)
#     sleep (measure_delay)


# p.channels[0].change_pmu_reg({
#     "CH EN" :0
# })


# k.smua.source.output = k.smua.OUTPUT_OFF
# df = pd.DataFrame({
#     "Measured Clamping " :  clamp_measured_values,
#     "Expected Clamping " :  clamp_expected_values
# })

 

# print(df)

Expected Clamping : 5.5684856414794925
Measured Clamping : 5.58021
Expected Clamping : 3.712273406982422
Measured Clamping : 3.70749
Expected Clamping : 1.8560611724853515
Measured Clamping : 1.83428
Expected Clamping : -0.00015106201171875
Measured Clamping : -0.0399075
Expected Clamping : -1.856363296508789
Measured Clamping : -1.91414
Expected Clamping : -3.712575531005861
Measured Clamping : -3.78847
Expected Clamping : -5.568787765502931
Measured Clamping : -5.6627
Expected Clamping : -7.4250000000000025
Measured Clamping : -7.53458
Expected Clamping : -9.281212234497072
Measured Clamping : -9.40817
Expected Clamping : -11.13742446899414
Measured Clamping : -11.2821
   Measured Clamping   Expected Clamping 
0            5.580210            5.568486
1            3.707490            3.712273
2            1.834280            1.856061
3           -0.039907           -0.000151
4           -1.914140           -1.856363
5           -3.788470           -3.712576
6           -5.662700     

In [5]:

# from keithley2600 import Keithley2600
# from pmu_som import communicator, pmu_ctrl, pmu_cal
# from time import sleep
# import numpy as np
# from matplotlib import pyplot as plt
# from tqdm import tqdm
# import pandas as pd


# try:
#     del(p.c)
# except:
#     pass

# k = Keithley2600('GPIB0::17::INSTR', visa_library='')
# #k.smua.source.output = k.smua.OUTPUT_OFF

# k.smua.reset()
# k.smub.reset()

# # p = communicator.communicator(False)
# p = pmu_ctrl.pmu()

# p.channels[0].change_pmu_reg({
#     "CH EN" : 0
# })

# k.display.screen = k.display.SMUA
# #k.display.smua.measure.func = k.display.MEASURE_DCVOLTS


# # k.smua.measure.nplc = 1
# # k.smua.measure.autorangei = k.smua.AUTORANGE_ON
# # k.smua.measure.autorangev = k.smua.AUTORANGE_ON


# # k.smua.source.limiti = ((2e-3)*2)
# # k.smua.source.limitv = 20
# # k.smua.source.func = k.smua.OUTPUT_DCAMPS
# # k.smua.source.levelv = 0
# # k.smua.source.output = k.smua.OUTPUT_ON



In [10]:
## Clamp Low


from keithley2600 import Keithley2600
from pmu_som import communicator, pmu_ctrl, pmu_cal
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd


try:
    del(p.c)
except:
    pass

k = Keithley2600('GPIB0::17::INSTR', visa_library='')
#k.smua.source.output = k.smua.OUTPUT_OFF

k.smua.reset()
k.smub.reset()

# p = communicator.communicator(False)
p = pmu_ctrl.pmu()

#print(k.query("*IDN?"))

p.channels[0].change_pmu_reg({
    "CH EN" : 0
})

p.write_all_PMU_REGS(0x1E060)

k.display.screen = k.display.SMUA
k.display.smua.measure.func = k.display.MEASURE_DCVOLTS


k.smua.measure.nplc = 1
k.smua.measure.autorangei = k.smua.AUTORANGE_ON
k.smua.measure.autorangev = k.smua.AUTORANGE_ON


k.smua.source.limiti = ((2e-3)*2)
k.smua.source.limitv = 20
k.smua.source.func = k.smua.OUTPUT_DCAMPS
k.smua.source.leveli = 0
k.smua.source.output = k.smua.OUTPUT_ON

# Configure the system control settings for the PMU
p.change_sys_ctrl({
    "DUTGND/CH": 1,
    "INT10K": 1,
    "GAIN": 2,
    "TMP ENABLE": 1,
    "TMP": 3  # Thermal shutdown at 100°C
})


# Enable and configure the first PMU channel
p.channels[0].change_pmu_reg({
    "CH EN": 1,  # Enable the channel
    "FORCE": 1,  # Force current clamp voltage
    "SS0": 1,
    "SF0": 1,
    "FIN": 1,
    "C": 3,
    "CL": 1
})

num = 10
overlap = 0.5
measure_points = np.linspace(round(0x7FFF+(0x8000*overlap)), 0, num)

CRF = 0.5
val = (2e-3)*-1*CRF
rsens = 500
MI_GAIN = 10
VREF = 4.95

DAC_C = round(((val * (2**16) * rsens * MI_GAIN)/(4.5 * VREF)) + (2**15))


measure_delay = 0.1


## This code is only for clamp high
## Writing the DAC code for 2mA v to FIN 1 2m

p.channels[0].write_dac(
    data = DAC_C,
    dac_reg = "FIN_I_2m",
    reg = "X1"
    
)


 ## Writing Clamp HIGH to FFFF

p.channels[0].write_dac(
    data = 0xFFFF,
    dac_reg = "CLH_U",
    reg = "X1"
)

dac_regi = "CLL_U"

clamp_measured_values = []
clamp_expected_values = []

for d in measure_points :
    p.channels[0].write_dac(
        data = round(d),
        dac_reg = dac_regi,
        reg = "X1"
    )
    sleep (measure_delay)
    clamp_measured = k.smua.measure.v()
    clamp_measured_values.append(clamp_measured)
    val = d
    offset_dac = 0xA492
    clamp_expected = ((4.5 * val) - (3.5 * offset_dac)) * (VREF / (2**16))
    clamp_expected_values.append(clamp_expected)
    
    print("Expected Clamping :",clamp_expected)
    print("Measured Clamping :",clamp_measured)
    sleep (measure_delay)


p.channels[0].change_pmu_reg({
    "CH EN" :0
})


k.smua.source.output = k.smua.OUTPUT_OFF

df = pd.DataFrame({
    "Measured Clamping " :  clamp_measured_values,
    "Expected Clamping " :  clamp_expected_values
})

 

print(df)
    

Expected Clamping : 5.5684856414794925
Measured Clamping : 5.5803
Expected Clamping : 3.712273406982422
Measured Clamping : 3.70748
Expected Clamping : 1.8560611724853515
Measured Clamping : 1.83428
Expected Clamping : -0.00015106201171875
Measured Clamping : -0.0399116
Expected Clamping : -1.856363296508789
Measured Clamping : -1.91415
Expected Clamping : -3.712575531005861
Measured Clamping : -3.78851
Expected Clamping : -5.568787765502931
Measured Clamping : -5.66273
Expected Clamping : -7.4250000000000025
Measured Clamping : -7.53456
Expected Clamping : -9.281212234497072
Measured Clamping : -9.40816
Expected Clamping : -11.13742446899414
Measured Clamping : -11.2821
   Measured Clamping   Expected Clamping 
0            5.580300            5.568486
1            3.707480            3.712273
2            1.834280            1.856061
3           -0.039912           -0.000151
4           -1.914150           -1.856363
5           -3.788510           -3.712576
6           -5.662730     

In [11]:
pmu_cal.measureCL_U(p, k, 0, range = "2m", current_range_factor=0.5, clamp_side='L', overlap=0.5, num=10, measure_delay = 0.1)

(array([ 5.56848564e+00,  3.71227341e+00,  1.85606117e+00, -1.51062012e-04,
        -1.85636330e+00, -3.71257553e+00, -5.56878777e+00, -7.42500000e+00,
        -9.28121223e+00, -1.11374245e+01]),
 array([  5.58005  ,   3.70747  ,   1.83424  ,  -0.0399272,  -1.91418  ,
         -3.78855  ,  -5.66273  ,  -7.53454  ,  -9.40818  , -11.2821   ]))